In [1]:
import pandas as pd
import numpy as np

## FX Overlay
## ----------------------------------------------------------------------------------
## 1. Loading data (GDP, Inflation, Futures, 2y yields)
## 2. Building THREE macro signals per region:
##      - Business Cycle   (GDP YoY – Inflation YoY)
##      - Monetary Policy  (– 12m change in 2y yield)
##      - Risk Sentiment   (12m equity futures return)
#       - Here we are missing the FX/ trade data 

## 3. Combines them into a single macro FX score per region & month

## 4. Standardizes scores cross-sectionally (z-scores across regions each month)

## 5. Maping the z-score into a Discrete Hedge Ratio:
##       strong positive -> 0% hedge (expect FX appreciation)
##       mid positive   -> 50% hedge
##       mid negative   -> 75% hedge
##       strong negative -> 100% hedge (expect FX depreciation)

## 6. Signal at t used for t+1 (Shifting the hedge ratio forward by one month)

## 7. Outputs a long-format table: Date, Region, FXScore_z, HedgeRatio_next_month



# --------------------------------------------------------------------------------------------
# 1. Load data from Excel
# --------------------------------------------------------------------------------------------

#  Adjust paths for your computer
gdp_path = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/GDP.xlsx"
inf_path = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/INF.xlsx"
fut_path = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/Futures.xlsx"
y2_path  = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/2_years_yields.xlsx"

# Quarterly GDP & Inflation (YoY) – one row per quarter
# Columns structure is: Date, CH_GDP, CH_GDP_YoY, JP_GDP, JP_GDP_YoY, ...
gdp = pd.read_excel(gdp_path, parse_dates=["Date"])
inf = pd.read_excel(inf_path, parse_dates=["Date"])

# Monthly equity futures – one row per month, proxy for equity prices
# Columns structure: Date, US, AU, CH, JP, UK, EM, EU
fut = pd.read_excel(fut_path, parse_dates=["Date"])

# Monthly 2-year yields – one row per month
# Columns structure: Date, US_PX_LAST, GB_PX_LAST, EU_PX_LAST, ...
y2  = pd.read_excel(y2_path, parse_dates=["Date"])

# Seting Date as index and sort chronologically.
# This ensures indexes are monotonic increasing
gdp = gdp.set_index("Date").sort_index()
inf = inf.set_index("Date").sort_index()
fut = fut.set_index("Date").sort_index()
y2  = y2.set_index("Date").sort_index()


In [ ]:
# ------------------------------------------------------------------------------------------------
# 2. Regions and column mapping
# --------------------------------------------------------------------------------------------------

# Labeles throughout the model.
canonical_regions = ["US", "GB", "EU", "JP", "CH", "EM", "AU"]

# Mapping between futures columns and region codes.
# Left = column name in Futures.xlsx
# Right = region code.
# Example: futures column "UK" corresponds to region "GB". Keeping GB notations
fut_col_map = {
    "US": "US",
    "AU": "AU",
    "CH": "CH",
    "JP": "JP",
    "UK": "GB",
    "EM": "EM",
    "EU": "EU",
}

# Mapping between 2-year yield columns and region codes.
# Left = column name in 2_years_yields.xlsx
# Right = canonical region code.
y2_col_map = {
    "US_PX_LAST": "US",
    "GB_PX_LAST": "GB",
    "EU_PX_LAST": "EU",
    "JP_PX_LAST": "JP",
    "CH_PX_LAST": "CH",
    "EM_PX_LAST": "EM",
    # When you add Australian 2y yields, add e.g.: "AU_PX_LAST": "AU"
}

def extract_region_from_col(col, suffix):
    """
    Helper function:
    Given a column name and a suffix, return the region code.

    Example:
        col    = "CH_GDP_YoY"
        suffix = "GDP_YoY"

    Then this returns "CH".
    If the column does not end with the suffix, return None.
    """
    if col.endswith(suffix):
        return col.split("_")[0]
    return None

In [5]:


# -------------------------
# 3. Create master monthly index
# -------------------------

# We want everything on a consistent **monthly** timeline at **month start dates**.
# We will use the futures data as our "master" monthly index (since it's monthly already).

# Make sure futures are sorted and then convert their index to month-start timestamps.
# Example: 1997-09-30 -> 1997-09-01, 1997-10-31 -> 1997-10-01, etc.
fut = fut.sort_index()
monthly_index = fut.index.to_period("M").to_timestamp()

# Very important: update fut.index to this month-start index.
# This way, fut now has one row per month, with date = YYYY-MM-01.
fut.index = monthly_index

# Do the same transformation for 2-year yields:
y2 = y2.sort_index()
y2.index = y2.index.to_period("M").to_timestamp()

# Now reindex yields to the same monthly_index and forward-fill any missing values.
# This ensures y2 and fut share exactly the same index.
y2 = y2.reindex(monthly_index).ffill()

# For GDP and Inflation (quarterly data):
# We reindex them to the monthly_index and forward-fill.
# That means each month between quarterly releases uses the latest available quarter.
gdp_monthly = gdp.reindex(monthly_index).ffill()
inf_monthly = inf.reindex(monthly_index).ffill()

# -------------------------
# 4. Build the three macro signals
# -------------------------

# 4.1 BUSINESS CYCLE SIGNAL:
#     BC = GDP YoY – Inflation YoY
#     Idea: strong real growth relative to inflation is good → positive signal.

bc_signal = pd.DataFrame(index=monthly_index,
                         columns=canonical_regions,
                         dtype=float)

for col in gdp_monthly.columns:
    # Look for columns ending with "GDP_YoY" like "CH_GDP_YoY"
    region = extract_region_from_col(col, "GDP_YoY")
    if region in canonical_regions:
        gdp_yoy = gdp_monthly[col]

        # Matching inflation YoY column: e.g. "CH_INF_YoY"
        inf_col = f"{region}_INF_YoY"
        if inf_col in inf_monthly.columns:
            inf_yoy = inf_monthly[inf_col]

            # Business cycle signal = growth minus inflation.
            bc_signal[region] = gdp_yoy - inf_yoy
        # If the inflation column is missing, we just leave that region as NaN.

# 4.2 MONETARY POLICY SIGNAL:
#     MP = − (12-month change in 2-year yield)
#     Idea:
#       - If 2-year yields fall (dy_12m < 0), central bank is easing → positive signal.
#       - If 2-year yields rise, central bank is tightening → negative signal.

mp_signal = pd.DataFrame(index=monthly_index,
                         columns=canonical_regions,
                         dtype=float)

for col, region in y2_col_map.items():
    if col in y2.columns and region in canonical_regions:
        y = y2[col]                 # 2-year yield series
        dy_12m = y - y.shift(12)    # 12-month change in yield
        mp_signal[region] = -dy_12m # Negate so that falling yields → positive signal

# 4.3 RISK SENTIMENT SIGNAL:
#     RS = 12-month equity futures return
#     Idea:
#       - Strong past 12m equity performance = strong sentiment → positive signal.
#       - Weak or negative past 12m performance = weak sentiment → negative.

rs_signal = pd.DataFrame(index=monthly_index,
                         columns=canonical_regions,
                         dtype=float)

for fut_col, region in fut_col_map.items():
    if fut_col in fut.columns and region in canonical_regions:
        p = fut[fut_col]                         # futures price
        rs_signal[region] = p / p.shift(12) - 1  # 12-month simple return

# -------------------------
# 5. Combine signals into a macro FX score
# -------------------------

# We now have three DataFrames:
#   bc_signal (Business Cycle)
#   mp_signal (Monetary Policy)
#   rs_signal (Risk Sentiment)
#
# Each has shape (months × regions).
# We want a single summary macro FX score per region and month:
#   FXScore_raw = average of available signals (BC, MP, RS).
# This mimics Brooks' equal-weighting across macro themes.

signal_stack = {
    "BC": bc_signal,
    "MP": mp_signal,
    "RS": rs_signal
}

fx_raw = pd.DataFrame(index=monthly_index,
                      columns=canonical_regions,
                      dtype=float)

for region in canonical_regions:
    parts = []

    # Collect this region's series from each theme.
    for _, df_sig in signal_stack.items():
        if region in df_sig.columns:
            parts.append(df_sig[region])

    if not parts:
        # If we somehow have no signals for this region, skip it.
        continue

    # Concatenate along columns: e.g. [BC, MP, RS], then take row-wise mean.
    stacked = pd.concat(parts, axis=1)

    # Equal-weighted average across the available signals for that month.
    fx_raw[region] = stacked.mean(axis=1, skipna=True)

# -------------------------
# 6. Cross-sectional z-score each month
# -------------------------

# Brooks standardizes signals cross-sectionally so that:
# - At each date, some regions are positive (above mean) and some negative (below mean).
# - Z-score: z_ij = (x_ij − mean_i) / std_i where i = date, j = region.
#   This ensures each month’s signal is centered and comparable across time.

def cross_sectional_zscore(df):
    """
    df: DataFrame of shape (time, regions)

    Returns:
        z-scored version where for each time t:
        z_tj = (x_tj - mean_t) / std_t
    """
    mean = df.mean(axis=1)
    std  = df.std(axis=1)
    z = df.sub(mean, axis=0).div(std, axis=0)
    return z

fx_z = cross_sectional_zscore(fx_raw)

# -------------------------
# 7. Discrete hedge ratio mapping (FX overlay rule)
# -------------------------

# We now have:
#   fx_z[t, region] = standardized macro FX score.
#
# Idea for FX overlay:
#   - Strong positive score → currency expected to appreciate → hedge less.
#   - Strong negative score → currency expected to depreciate → hedge more.
#
# We map z-score to discrete hedge ratios:
#   z >  1.0        → HedgeRatio = 0.00  (0% hedged, fully exposed to FX)
#   0 < z ≤ 1.0     → HedgeRatio = 0.50  (50% hedged)
#  -1.0 ≤ z ≤ 0     → HedgeRatio = 0.75  (75% hedged)
#   z < -1.0        → HedgeRatio = 1.00  (100% hedged, fully hedge FX)

def hedge_ratio_from_score(z):
    """
    Map z-scores to discrete hedge ratios between 0 and 1.

    z: DataFrame with same shape as fx_z (time x region)
    """
    hr = pd.DataFrame(index=z.index, columns=z.columns, dtype=float)

    for region in z.columns:
        s = z[region]

        hr_region = np.select(
            [
                s > 1.0,
                (s > 0.0) & (s <= 1.0),
                (s > -1.0) & (s <= 0.0),
                s <= -1.0,
            ],
            [
                0.0,   # strong positive -> 0% hedge
                0.5,   # mild positive   -> 50% hedge
                0.75,  # mild negative   -> 75% hedge
                1.0,   # strong negative -> 100% hedge
            ],
            default=np.nan,  # if z is NaN
        )

        hr[region] = hr_region

    return hr

hedge_ratio = hedge_ratio_from_score(fx_z)

# -------------------------
# 8. Shift so signal at t is used to hedge in t+1
# -------------------------

# When you implement the strategy, you only know the signal at the end of month t,
# so you must apply it to the hedge decision for month t+1.
# Therefore we shift the hedge ratio forward by 1 month.

hedge_ratio_next = hedge_ratio.shift(1)

# -------------------------
# 9. Final long-format table
# -------------------------

# fx_z: wide format (time x region)
# hedge_ratio_next: same shape
# We convert to long format with one row per (Date, Region).

fx_z_long = fx_z.stack().rename("FXScore_z")                  # MultiIndex: (Date, Region)
hedge_long = hedge_ratio_next.stack().rename("HedgeRatio_next_month")

# Combine into a single DataFrame
result = pd.concat([fx_z_long, hedge_long], axis=1).reset_index()
result.columns = ["Date", "Region", "FXScore_z", "HedgeRatio_next_month"]

# Save to CSV so you can inspect and use it in other parts of your project
result.to_csv("fx_hedge_signals_discrete.csv", index=False)

# Show first 20 lines as a quick check
print(result.head(20))


         Date Region  FXScore_z  HedgeRatio_next_month
0  1998-09-01     US   1.414493                    NaN
1  1998-09-01     GB   0.945184                    NaN
2  1998-09-01     EU   0.551510                    NaN
3  1998-09-01     JP  -1.195396                    NaN
4  1998-09-01     CH  -0.407623                    NaN
5  1998-09-01     EM  -0.235222                    NaN
6  1998-09-01     AU  -1.072946                    NaN
7  1998-10-01     US   0.969936                   0.00
8  1998-10-01     GB   0.844101                   0.50
9  1998-10-01     EU   0.691601                   0.50
10 1998-10-01     JP  -1.326462                   1.00
11 1998-10-01     CH   0.318408                   0.75
12 1998-10-01     EM  -0.083156                   0.75
13 1998-10-01     AU  -1.414428                   1.00
14 1998-11-01     US   0.653641                   0.50
15 1998-11-01     GB   1.386619                   0.50
16 1998-11-01     EU   0.646681                   0.50
17 1998-11

In [8]:
bc_signal = bc_signal.sort_values("Date", ascending=False)#.reset_index(drop=True)
mp_signal = mp_signal.sort_values("Date", ascending=False)
rs_signal = rs_signal.sort_values("Date", ascending=False)

print(bc_signal)
print(mp_signal)
print(rs_signal)

             US   GB   EU   JP   CH   EM   AU
Date                                         
2025-10-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-09-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-08-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-07-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-06-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...         ...  ...  ...  ...  ...  ...  ...
1998-01-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-12-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-11-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-10-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-09-01  NaN  NaN  NaN  NaN  NaN  NaN  NaN

[338 rows x 7 columns]
                US     GB     EU     JP     CH      EM  AU
Date                                                      
2025-10-01  0.5966  0.666  0.313 -0.479  0.528  0.4895 NaN
2025-09-01  0.0328 -0.002  0.049 -0.552  0.532  0.1547 NaN
2025-08-01  0.2998  0.166  0.451 -0.497  0.693  0.3335 NaN
2025-07-01  0.3004 -0.035  0.567 -0.367  0.742  0.5227 NaN
2025-06-01  1.0343  0.40